In [1]:
# Importing the needed Dependencies
import requests
import pymongo
import pandas as pd
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo



In [2]:
# Setting up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/cynthia10/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


### NASA Mars News

In [3]:
# Defining the url to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)

time.sleep(1)

In [4]:
# Scraping the url page into Soup
html=browser.html

# Parsing with beautiful soup parser
soup = bs(html, "html.parser")

In [5]:
# Retrieving the latest News Title
news_title=soup.find_all('div', class_='content_title')[0].text
news_title

"NASA Wins 4 Webbys, 4 People's Voice Awards"

In [6]:
# Retrieving the latest news Paragraph Text
news_p=soup.find_all('div', class_="article_teaser_body")[0].text
news_p

'Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA\'s Global Climate Change website and Solar System Interactive.'

### JPL Mars Space Images - Featured Image

In [7]:
# Obtaining the url for the featured space image site
# featured_url = "https://redplanetscience.com/"
# featured_image_url = "https://mars.nasa.gov/system/news_items/list_view_images/8567_PIA23517-320x240.jpg"
#browser.visit(featured_image_url)

base_url = "https://spaceimages-mars.com/"
browser.visit(base_url)
time.sleep(3)

In [8]:
# Defining the HTML object
html = browser.html

In [9]:
# Parsing the HTML
soup=bs(html,"html.parser")

In [10]:
# Retrieve image from the url
image_url=soup.find("img", class_ = "headerimage fade-in")["src"]
image_url

'image/featured/mars3.jpg'

In [11]:
featured_image_url = base_url + image_url
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


### Mars Fact

In [12]:
# Scraping Mars facts from galaxyfacts website
url='https://galaxyfacts-mars.com/'
html_tables=pd.read_html(url)


In [13]:
# Converting the data into a dataframe
mars_fact = html_tables[0]
mars_fact_df = mars_fact.rename(columns={0:"Profile",1:"Value"},errors="raise")

mars_fact_df.set_index("Profile",inplace=True)
mars_fact_df

,Value,2
Profile,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
# Converting the table to html
mars_fact_table=mars_fact_df.to_html()

In [15]:

mars_fact_table.replace('\n','')
print(mars_fact_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
      <th>2</th>
    </tr>
    <tr>
      <th>Profile</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Mars - Earth Comparison</th>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [16]:
#Navigate to url for scraping
base_url  = "https://marshemispheres.com/"
browser.visit(base_url)
time.sleep(3)

#Container to hold  our loop iterations
hemisphere_image_urls  = []

#Iterate through the 4 different links to get the images needed
for item in range(4):
    html = browser.html
    soup = bs(html, "html.parser")
    
    title = soup.find_all("h3")[item].get_text()
    browser.find_by_tag("h3")[item].click()
    
    html = browser.html
    soup = bs(html, "html.parser")
    
    hemisphere_url = soup.find("img", class_="wide-image")["src"]
    hemisphere_image_urls.append({
        "title":title,
        "img_url": base_url+hemisphere_url
    })
 
    browser.back()
hemisphere_image_urls 

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]